In [49]:
#converts input image to ela applied image
from PIL import Image, ImageChops, ImageEnhance


def convert_to_ela_image(filename,quality):

    Original_Image = Image.open(filename).convert('RGB')

    Compressed_Image_filename = 'resaved_image.jpg'
    Original_Image.save(Compressed_Image_filename,'JPEG',quality=quality)
    Compressed_Image = Image.open(Compressed_Image_filename)

    ELA_Image = ImageChops.difference(Original_Image,Compressed_Image)
    
    #scaling factors are calculated from pixel extremas
    extrema = ELA_Image.getextrema()
    max_difference = max([pix[1] for pix in extrema])
    if max_difference ==0:
        max_difference = 1
    scale = 350.0 / max_difference
    
    ELA_Image = ImageEnhance.Brightness(ELA_Image).enhance(scale)

    ELA_Image.save("ela_image.png")
    return ELA_Image

In [54]:
from os import listdir
from os.path import isfile

import numpy as np

def PreProcess(image_path):
    return np.array(convert_to_ela_image(image_path, 90).resize((128, 128))).flatten() / 255.0

def getDataSet(DataSetType):
    X = [] # Images
    Y = [] # false = fake, true = real

    if(DataSetType == 0):
        TestOrTrain = 'train'
    else:
        TestOrTrain = 'test'

    path = TestOrTrain+'/REAL'
    print(path);

    allFiles = []
    for f in listdir(path):
        if(isfile(path + '/'+ f)):
            allFiles.append(path + '/'+ f);
    
    for file in allFiles:
        if file.endswith('jpg') or file.endswith('png'):
            X.append(PreProcess(file))        
            Y.append(True)     # label for authentic images 
            
    print(f'Total images: {len(X)}\nTotal labels: {len(Y)}')


    path = TestOrTrain+'/FAKE'
    print(path);

    allFiles = []
    for f in listdir(path):
        if(isfile(path + '/'+ f)):
            allFiles.append(path + '/'+ f);
    
    for file in allFiles:
        if file.endswith('jpg') or file.endswith('png'):
            X.append(PreProcess(file))        
            Y.append(False)     # label for authentic images 
            
    print(f'Total images: {len(X)}\nTotal labels: {len(Y)}')

    X = np.array(X)
    Y = Y.astype(np.bool_)
    Y = np.packbits(Y, axis=-1)
    X = X.reshape(-1, 128, 128, 3)

    return X, Y

In [ ]:
test_X, test_Y = getDataSet(1);


test/REAL
Total images: 10000
Total labels: 10000
test/FAKE
